In [1]:
#
# Initialization
#
import os
import sys

import ipynbname
from pathlib import Path

# Set notebook's src module path. Note that you may have to update your IDE's project settings to do the same for the
#  local library imports to work the same

MODULE_PATH = ipynbname.path().parent.parent
sys.path.append(str(MODULE_PATH))

# Keep paths consistent throughout notebook
os.chdir(MODULE_PATH)

# This should always be `./src`
print(f"Current working directory [{os.getcwd()}]")

# Place all local artifacts in a disposable, git-ignored directory
local_artifact_dir = Path(os.getcwd()).parent / "out"
local_artifact_dir.mkdir(parents=True, exist_ok=True)

# Autoreload imports at the beginning of cell execution.
#  https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

Current working directory [/Users/jbeckman/projects/capia/src]


In [2]:
#
# Setup utils
#

from utils.logger_util import LoggerUtil
from utils.utils import Utils
from utils import config

LOGGER = LoggerUtil(config.MODEL_ID, local_artifact_dir / "logs")
UTILS = Utils(LOGGER)

UTILS.describe_env()

AWS_INSTANCE = 'ml.m5.large' # 2 vCPU, 0 GPU, 8 GB memory, $0.134/hour
LOCAL_INSTANCE = 'local'

INSTANCE_TYPE = LOCAL_INSTANCE
IS_LOCAL = LOCAL_INSTANCE == INSTANCE_TYPE

/Users/jbeckman/projects/capia/venv/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


2021-12-02 15:42:06.843416 Background logger started
2021-12-02 15:42:06.843915 The model id is [giia-1.0.3]
2021-12-02 15:42:06.844014 The MXNet version is [1.8.0]
2021-12-02 15:42:06.844090 The GluonTS version is [0.8.1]
2021-12-02 15:42:06.844161 The SageMaker version is [2.70.0]
2021-12-02 15:42:06.844271 The GPU count is [0]


In [3]:
from data_processing.aws_handler import AWSHandler
from sagemaker import LocalSession

AWS_HANDLER = AWSHandler(LOGGER, config.MODEL_ID)


if IS_LOCAL:
    LOGGER.log("Notebook is set to local mode, not uploading to S3")

    sagemaker_session = LocalSession()
    sagemaker_session.config = {
        'local': {
            'local_code': True
        }
    }
else:
    sagemaker_session = AWS_HANDLER.sagemaker_session

2021-12-02 15:42:10.111981 Notebook is set to local mode, not uploading to S3


In [4]:
#
# Upload serialized model
#

from sagemaker.mxnet import MXNetModel

# Place the model (`model.tar.gz` contents) that you would like to run locally inside this directory.
# This will recompress them in the appropriate format
model_dir_path = local_artifact_dir / "local_deploy_model" / "model"
model_dir_path.mkdir(parents=True, exist_ok=True)
LOGGER.log(f"Model dir is [{model_dir_path}]")

# This is stupid, but we have to upload the model to S3 for Sagemaker, even though Sagemaker will download it locally
# anyways to run it. Their Model class does not support local paths
s3_model_path = AWS_HANDLER.upload_model_to_s3(model_dir_path)

2021-12-02 15:42:13.419307 Model dir is [/Users/jbeckman/projects/capia/out/local_deploy_model/model]
2021-12-02 15:42:13.419666 Compressing [/Users/jbeckman/projects/capia/out/local_deploy_model/model] to [/Users/jbeckman/projects/capia/out/local_deploy_model/model.tar.gz]
2021-12-02 15:42:13.940077 Compress complete
2021-12-02 15:42:13.941275 Uploading [/Users/jbeckman/projects/capia/out/local_deploy_model/model.tar.gz] to [s3://sagemaker-us-east-1-941048668662/local_deploy_model/model.tar.gz]
2021-12-02 15:42:14.704320 Upload complete


In [11]:
#
# Deploy model
#

LOGGER.log(f"Removing preexisting models and endpoints")
try:
    sagemaker_session.sagemaker_client.delete_model(config.MODEL_NAME)
    sagemaker_session.sagemaker_client.delete_endpoint(config.MODEL_NAME)
    sagemaker_session.sagemaker_client.delete_endpoint_config(config.MODEL_NAME)
except:
    pass

LOGGER.log(f"Creating model")
model = MXNetModel(
    model_data=s3_model_path,
    name=config.MODEL_NAME,
    entry_point='sff.py',
    source_dir=os.getcwd(),
    role=config.SM_ROLE,
    framework_version='1.8.0',  # Should be the same mxnet X.X.X version found in requirements.txt
    py_version='py37',
    sagemaker_session=sagemaker_session,
    # container_log_level=logging.DEBUG
)

LOGGER.log(f"Deploying model")
predictor = model.deploy(
    instance_type=INSTANCE_TYPE,
    endpoint_name=config.MODEL_NAME,
    initial_instance_count=1)
LOGGER.log(sagemaker_session.sagemaker_client.describe_model(config.MODEL_NAME))
LOGGER.log(sagemaker_session.sagemaker_client.describe_endpoint_config(config.MODEL_NAME))
LOGGER.log(f"Predictor metadata [{predictor.__dict__}]")

if IS_LOCAL:
    LOGGER.log(f"http://localhost:{sagemaker_session.sagemaker_runtime_client.serving_port}/invocations")
else:
    LOGGER.log(f"http://???:{sagemaker_session.sagemaker_runtime_client.serving_port}/invocations")


2021-12-02 16:14:00.612533 Removing preexisting models and endpoints
2021-12-02 16:14:00.643743 Creating model
2021-12-02 16:14:00.644802 Deploying model


ValueError: Unsupported Python version: py38. You may need to upgrade your SDK version (pip install -U sagemaker) for newer Python versions. Supported Python version(s): py37.